# Introduction to BIDS and `PyBIDS`
[Brain Imaging Data Structure (**BIDS**)](https://bids-specification.readthedocs.io/en/stable/) is a a standard for organizing and describing neuroimaging (and behavioural) datasets. See [BIDS paper](https://doi.org/10.1038/sdata.2016.44) and http://bids.neuroimaging.io website for more information.

`PyBids` is a Python module to interface with datasets conforming BIDS. See the [documentation](https://bids-standard.github.io/pybids/) and [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7409983/) for more info. `PyBids` can be installed with `pip install pybids` command.

**Here we will explore some of the functionality of pybids.layout.** The material is adapted from https://github.com/bids-standard/pybids/tree/master/examples

**First you will need to download the example dataset by executing the cell below:**

In [1]:
print("Downloding the example dataset ... please wait ... ")
![ ! -d "FaceRecognition" ] && wget "FaceRecognition.zip" "https://dl.dropboxusercontent.com/s/q030cu844joczm6/FaceRecognition.zip"
![ ! -d "FaceRecognition" ] && unzip -q ~/hands-on/FaceRecognition.zip -d ~/hands-on && rm ~/hands-on/FaceRecognition.zip 
print("Dataset downloaded, carry on!")

Downloding the example dataset ... please wait ... 
Dataset downloaded, carry on!


## The `BIDSLayout`
A `BIDSLayout` is a Python class that represents a BIDS project file tree and provides a variety of helpful methods for querying and manipulating BIDS files. The `BIDSLayout` initializer has a large number of arguments you can use to control the way files are indexed and accessed. But usually you'd initialise a `BIDSLayout` with just its root path location.

In [ ]:
import os
from bids.layout import BIDSLayout

ds_path = 'FaceRecognition/data/bids'

# Initialize the layout
layout = BIDSLayout(ds_path)

# Print some basic information about the layout
layout

### Querying the BIDSLayout
The main method for querying `BIDSLayout` is `.get()`.

If we call `.get()` with no additional arguments, we get back a list of all the BIDS files in our dataset.

In [ ]:
all_files = layout.get()
print("There are {} files in the layout.".format(len(all_files)))
print("\nThe first 5 files are:")
all_files[:5]

The returned object is a **Python list**. Each element in the list is a `BIDSFile` object. 

We can also get just filenames.

In [ ]:
layout.get(return_type='filename')[:5]

We can also get such information as
* all `subject` IDs
* all `task` names
* dataset `description`
* the BOLD repetition time TR
* how many `runs` there are

Get a list of all subject IDs. 

In [ ]:
# write your code here


In [ ]:
layout.get_subjects()

Get a list of all tasks in this dataset

In [ ]:
# write your code here


In [ ]:
layout.get_tasks()

Get description of the dataset

In [ ]:
# write your code here


In [ ]:
layout.get_dataset_description()

Get the repetition time of the functional runs

In [ ]:
# write your code here


In [ ]:
layout.get_tr()

Get functional run numbers for each subject

In [ ]:
# write your code here


In [ ]:
for sID in layout.get_subjects(): 
    print(layout.get_runs(subject = sID))

### Filtering files by entities
We can pass any BIDS-defined entities (keywords) to `.get()` method. 

Retrieve all BOLD runs with `.nii.gz` extensions for subject `04`.

In [ ]:
# write your code here


In [ ]:
# Retrieve filenames of all BOLD runs for subject 01
layout.get(subject='04', extension='nii.gz', suffix='bold', return_type='filename')

All of the entities are found in the names of BIDS files. For example `sub-01_task-facerecognition_run-01_bold.nii.gz` has entities: **subject** **task** **run** **suffix** **extension**.

You can get the list of all availabe entities by `layout.get_entities()`.

Here are a few of the most common entities:

* `suffix`: The part of a BIDS filename just before the extension (e.g., 'bold', 'events', 'T1w', etc.).
* `subject`: The subject label
* `session`: The session label
* `run`: The run index
* `task`: The task name

### Filtering by metadata
Sometimes we want to search for files based not just on their names, but also based on metadata defined in JSON files. We can pass any key that occurs in any JSON file in our project as an argument to `.get()`. We can combine these with any number of core BIDS entities (like `subject`, `run`, etc.).

Write a code to retrieve `SpacingBetweenSlices` for all subjects. And create a nice data frame of this information.

In [ ]:
# write your code here


In [ ]:
import pandas as pd
d = []
for subject in layout.get_subjects():
    d.append(
        {
            'subject': subject,
            'spacing': layout.get_SpacingBetweenSlices(subject=subject, suffix='bold')
        }
    )
df = pd.DataFrame(d)
df = df.sort_values(by=['subject'])

print(df.to_string(index=False))

### Other `return_type` values
We can also ask `get()` to return unique values (or IDs) of particular entities. For example, we want to know which subjects have at least one `T1w` file. We can request that information by setting `return_type='id'`. When using this option, we also need to specify a target entity (or metadata keyword) called `target`. This combination tells the `BIDSLayout` to return the unique values for the specified `target` entity. 

Ask for all of the unique subject IDs that have at least one file with a `phasediff` suffix. 

In [ ]:
# write your code here


In [ ]:
layout.get(return_type='id', target='subject', suffix='phasediff')

If our `target` is a BIDS entity that corresponds to a particular directory in the BIDS specification (e.g., `subject` or `session`) we can also use `return_type='dir'` to get all matching subdirectories. 

In [ ]:
layout.get(return_type='dir', target='subject')

## The `BIDSFile`
When you call `.get()` on a `BIDSLayout`, the default returned values are objects of class `BIDSFile`. A `BIDSFile` is a lightweight container for individual files in a BIDS dataset. It provides easy access to a variety of useful attributes and methods. Let's take a closer look. First, let's pick a random file from our existing `layout`.

In [ ]:
# Pick the 7th file in the dataset
bf = layout.get()[7]
# Print it
bf

Here are some of the attributes and methods available to us in a `BIDSFile` (note that some of these are only available for certain subclasses of `BIDSFile`; e.g., you can't call `get_image()` on a `BIDSFile` that doesn't correspond to an image file!):
* `.path`: The full path of the associated file
* `.filename`: The associated file's filename (without directory)
* `.dirname`: The directory containing the file
* `.get_entities()`: Returns information about entities associated with this `BIDSFile` (optionally including metadata)
* `.get_image()`: Returns the file contents as a nibabel image (only works for image files)
* `.get_df()`: Get file contents as a pandas DataFrame (only works for TSV files)
* `.get_metadata()`: Returns a dictionary of all metadata found in associated JSON files
* `.get_associations()`: Returns a list of all files associated with this one in some way

Let's see some of these in action.

In [ ]:
# Print all the entities associated with this file, and their values


In [ ]:
bf.get_entities()

In [ ]:
# Print all the metadata associated with this file


In [ ]:
bf.get_metadata()

In [ ]:
# We can the union of both of the above in one shot like this
bf.get_entities(metadata='all')

Here are all the files associated with our target file in some way.

In [ ]:
bf.get_associations()

`.get_image()`: Returns the file contents as a `nibabel` image (only works for image files). We can then display the image, for example, using `OrthoSlicer3D` which requires `matplotlib`.   

**Note:** When using `orthoview()` in notebook, don't forget to close figures afterward again or use `%matplotlib inline` again, otherwise, you cannot plot any other figures.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

bf.get_image().orthoview()

In [ ]:
# Pick the 18th file in the dataset
# Assign it a variable name b18
# Display the file with orthoview




In [ ]:
bf18 = layout.get()[18]
bf18.get_image().orthoview()

## `.tsv` files

In cases where a file has a `.tsv.gz` or `.tsv` extension, it will automatically be created as a `BIDSDataFile`, and we can easily grab the contents as a `DataFrame`.

Let's look at the first `events` file from our layout.

In [ ]:
# Get the first events file
evfile = layout.get(suffix='events')[0]

# Get contents as a DataFrame and show the first few rows
df = evfile.get_df()
df.head()

Let's look at the `participants` information. 

In [ ]:
participants = layout.get(suffix='participants', extension='tsv')[0]
participants.get_df()

## Other utilities

### Filename parsing
Let's say you have a filename, and you want to manually extract BIDS entities from it. The parse_file_entities method provides the facility:

In [ ]:
layout.parse_file_entities('some_path_to_bids_file/sub-04_task-facerecognition_run-01_bold.nii.gz')

You can do the same for `BIDSFile` object that we defined earlier. How would you do it for either `bf` or `bf18`?

In [ ]:
# write your code here


In [ ]:
layout.parse_file_entities(bf.path)

### Path construction
You may want to create valid BIDS filenames for files that are new or hypothetical that would sit within your BIDS project. This is useful when you know what entity values you need to write out to, but don’t want to deal with looking up the precise BIDS file-naming syntax. In the example below, imagine we’ve created a new file containing stimulus presentation information, and we want to save it to a `.tsv.gz` file, per the BIDS naming conventions. All we need to do is define a dictionary with the name components, and build_path takes care of the rest (including injecting sub-directories!):

In [ ]:
entities = {
    'subject': '01',
    'run': '02',
    'task': 'facerecognition',
    'suffix': 'bold'
}

layout.build_path(entities)

Keep in mind that `_` and `-` have special meaning in BIDS specification. E.g., you can't name your task `face-recognition`, that would not be 'spec-compliant' and would end in an error! However, if you add `validate=False`, you can get away with it (i'e', `layout.build_path(entities, validate=False)`). 

You can also use `build_path` in more sophisticated ways by defining your own set of matching templates that cover cases not supported by BIDS out of the box. For example, suppose you want to create a template for naming a new `stat` file. You could do something like:

In [ ]:
# Define the pattern to build out of the components passed in the dictionary
pattern = "sub-{subject}[_ses-{session}]_task-{task}[_run-{run}]_{suffix}.nii.gz"

entities = {
    'subject': '01',
    'run': '02',
    'task': 'facerecognition',
    'suffix': 'stat'
}

# Notice we pass the new pattern as the second argument
layout.build_path(entities, pattern, validate=False)

### Exporting a `BIDSLayout` to a pandas `Dataframe`
If you want a summary of all the files in your `BIDSLayout`, but don't want to have to iterate `BIDSFile` objects and extract their entities, you can get a nice bird's-eye view of your dataset using the `to_df()` method.

In [ ]:
# Convert the layout to a pandas dataframe
df = layout.to_df()
df.head()

We can also include metadata in the result if we like (which may blow up our `DataFrame` if we have a large dataset). Note that in this case, most of our cells will have missing values.

In [ ]:
df = layout.to_df(metadata=True)
df.head()

## Report generation
`PyBIDS` also allows you to automatically create data acquisition reports based on the available `image` and `meta-data` information. This enables a new level of standardisation and transparency. FAIR-ness, meta-analyses, etc. 

In [ ]:
# import the BIDSReport function from the reports submodule
from bids.reports import BIDSReport

Now we only need to apply the `BIDSReport` function to our `layout` and generate our report. 

In your example dataset the report might run into error as the dataset is incomplete. The code below deals with that case. But a successfull report would look something like this:

>In session None, MR data were acquired using a 3-Tesla Siemens TrioTim MRI scanner.
	One run of T1-weighted SP\MP\OSP GR\IR (GR\IR) single-echo structural MRI data were collected (256 slices; repetition time, TR=2250ms; echo time, TE=2.98ms; flip angle, FA=9<deg>; field of view, FOV=192x256mm; matrix size=192x256; voxel size=1x1x1mm).
	A spoiled gradient recalled (GR) field map (phase encoding: anterior to posterior; 33 slices in interleaved ascending order; repetition time, TR=400ms; echo time 1 / 2, TE1/2=5.197.65ms; flip angle, FA=60<deg>; field of view, FOV=192x192mm; matrix size=64x64; voxel size=3x3x3.75mm) was acquired for the first, second, third, fourth, fifth, sixth, seventh, eighth, and ninth runs of the facerecognition BOLD scan.
	Nine runs of facerecognition segmented k-space echo planar (EP) single-echo fMRI data were collected (33 slices in interleaved ascending order; repetition time, TR=2000ms; echo time, TE=30ms; flip angle, FA=78<deg>; field of view, FOV=192x192mm; matrix size=64x64; voxel size=3x3x3.75mm). Run duration was 6:56 minutes, during which 208 volumes were acquired.

>Dicoms were converted to NIfTI-1 format using dcm2niix (v1.0.20220720). This section was (in part) generated automatically using pybids (0.15.3).

In [ ]:
# Initialize a report for the dataset
report = BIDSReport(layout)

# Method generate returns a Counter of unique descriptions across subjects
try:
    descriptions = report.generate()
    pub_description = descriptions.most_common()[0][0]
    print(pub_description)
except IndexError:
    print('Sorry, it seems that the dataset is not complete and report cannot be generated.')